In [2]:
import nltk
import glob
import os
import re
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
import pickle5 as pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rupin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rupin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
PATH = 'CSE508_Winter2023_Dataset/CSE508_Winter2023_Dataset'
PATH_upd = 'CSE508_Winter2023_Dataset_upd/CSE508_Winter2023_Dataset'
flist = os.listdir(PATH)

In [4]:
def rem_sw(arr):
  global stop_words
  reg_str = r'^'
  for i in range(len(arr)):
    for sw in stop_words:
      reg_str+=sw
      reg_str+='$'
      arr[i] = re.sub(reg_str, '', arr[i])
      reg_str = r'^'
  return arr

def rem_punc(str):
  str = re.sub(r'(\.|\?|\,|!|\:|\;|\&|\-|\(|\)|\{|\}|\'|\"|\/)', ' ', str)
  return str

In [5]:
### biword index creation ###
biword_index_dict = {}
def bigram_inv_index(file_data, file_id):
        file_data_list = file_data.split(",")
        # print(file_data_list)
        i=0
        while(i<len(file_data_list)-1):
          biword_term = file_data_list[i]+" "+file_data_list[i+1]

          if biword_term not in biword_index_dict:
            biword_index_dict[biword_term] = []
            biword_index_dict[biword_term].append(file_id)

          elif file_id in biword_index_dict[biword_term]:
            pass
          
          else:
            biword_index_dict[biword_term].append(file_id)

          i+=1

In [6]:
import os
directory = PATH_upd
file_no = 0
all_files = {}           # stores the (file names : file_no )
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):

        file = open(f,errors="ignore") 
        bigram_inv_index(file.read(),file_no)
        all_files[file_no] = filename
        file_no += 1

        file.close()

In [ ]:
# pos_index structure: {term : [df, {doc_id : [tf, [pos1, pos2, ...]]}]} df = frequency in all docs combined, tf = frequency in a particular doc, pos[] = positions of term in doc
pos_index = {}
for doc_id in flist:
    f = open(PATH_upd+doc_id, 'r')
    tokens = f.read().split(',')
    for num,j in enumerate(tokens):
        if j not in set(pos_index.keys()):  #new term
            pos_index[j] = []
            pos_index[j].append(1)
            pos_index[j].append({doc_id:[1,[num]]})
        else:
            pos_index[j][0]+=1
            if doc_id not in set(pos_index[j][1].keys()):   #new document for this term
                pos_index[j][1][doc_id] = [1]
                pos_index[j][1][doc_id].append([num])
            else:
                pos_index[j][1][doc_id][1].append(num)
                pos_index[j][1][doc_id][0]+=1
    f.close()


In [ ]:
with open('pos_index.pickle', 'wb') as f:
    pickle.dump(pos_index, f, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
with open('pos_index.pickle', 'rb') as f:
    pos_index = pickle.load(f)

In [8]:
all_files[0]

'cranfield0001'

In [9]:
def eval_pos_query(query):
    ans_docs = {}   # stores the possible documents with list of possible phrase start positions
    for n, term in enumerate(query):
        if n==0:    # for the first term
            #ans_docs = pos_index[term][1]
            for i in list(set(pos_index[term][1].keys())):
                ans_docs[i] = pos_index[term][1][i][1].copy()
            #print(ans_docs['cranfield1201'])
            #for i in list(set(pos_index[term][1].keys())):
                #ans_docs[i] = pos_index[term][1][i]
        else:
            to_check_docs = list(set(pos_index[term][1].keys()).intersection(set(ans_docs.keys())))
            #print(to_check_docs)
            for doc in to_check_docs:
                temp = []
                for pos in pos_index[term][1][doc][1]:
                    if pos-n in set(ans_docs[doc]):
                        temp.append(pos-n)
                        #ans_docs[doc].remove(pos-n)
                ans_docs[doc] = [i for i in ans_docs[doc] if i in set(temp)]
                    
                if len(ans_docs[doc]) ==0:  # if no possible phrase start positions left then remove the document
                    del ans_docs[doc]
            for doc in list(set(ans_docs.keys()).difference(set(to_check_docs))): # remove documents which do not contain the term
                del ans_docs[doc]
            if len(ans_docs) == 0:
                return ans_docs
    return ans_docs
    

In [18]:
import pickle

with open('biword_dict.pickle', 'wb') as f:
    pickle.dump(biword_index_dict, f, protocol=pickle.HIGHEST_PROTOCOL)



In [19]:
with open('biword_dict.pickle', 'rb') as f:
    biword_index_dict = pickle.load(f)

In [11]:
def intersection(list1, list2):
    list3=[]
    for value in list1:
      if(value in set(list2)):
        list3.append(value)
    
    return list3

In [12]:
def queryPreprocess(s):
  s = s.lower()
  s = re.sub(r'(\.|\?|\,|!|\:|\;|\&|\-|\(|\)|\{|\}|\'|\"|\/)', ' ', s)
  s = word_tokenize(s)
  s = rem_sw(s)
  for j in range(len(s)):
    s[j] = rem_punc(s[j])
  s[:] = [j for j in s if (j!='' and j!=' ')]
  return s

In [23]:
n=int(input("Enter number of queries: "))
for i in range(n):
  q=input("Enter query")
  query = queryPreprocess(q)  #returns tokenized processed query
  print(query)
  pos_ans = eval_pos_query(query)
  list_=[]
  for j in range(len(query)):
    if(query[j]!=''):
      list_.append(query[j])
  biwords=[]
  for j in range(len(list_)-1):
    biwords.append(list_[j]+" "+list_[j+1])


  ans=[]
  for i in range(1400):# initial list of all files
    ans.append(i)

  for i in range(len(biwords)):
    term=biwords[i]
    if(term in biword_index_dict.keys()):
      ans=intersection(ans,biword_index_dict[term])
    else:
      ans=[]
      break
  for j in range(len(ans)):
    ans[j]=all_files[ans[j]]
  if len(query)==1:
    ans = []
  
  ans.sort()
  print(f"Number of documents retrieved for query {i+1} using bigram inverted index: {len(ans)}")
  print(f"Names of documents retrieved for query {i+1} using bigram inverted index: {ans}")
  print(f"Number of documents retrieved for query {i+1} using positional index: {len(pos_ans)}")
  print(f"Names of documents retrieved for query {i+1} using positional index: {sorted(list(pos_ans.keys()))}")


['experimental']
Number of documents retrieved for query 1400 using bigram inverted index: 0
Names of documents retrieved for query 1400 using bigram inverted index: []
Number of documents retrieved for query 1400 using positional index: 318
Names of documents retrieved for query 1400 using positional index: ['cranfield0001', 'cranfield0011', 'cranfield0012', 'cranfield0017', 'cranfield0019', 'cranfield0025', 'cranfield0029', 'cranfield0030', 'cranfield0035', 'cranfield0041', 'cranfield0042', 'cranfield0047', 'cranfield0052', 'cranfield0053', 'cranfield0058', 'cranfield0069', 'cranfield0070', 'cranfield0074', 'cranfield0078', 'cranfield0084', 'cranfield0099', 'cranfield0101', 'cranfield0103', 'cranfield0112', 'cranfield0115', 'cranfield0121', 'cranfield0123', 'cranfield0137', 'cranfield0140', 'cranfield0142', 'cranfield0154', 'cranfield0156', 'cranfield0168', 'cranfield0170', 'cranfield0171', 'cranfield0173', 'cranfield0176', 'cranfield0179', 'cranfield0183', 'cranfield0184', 'cranfiel